### Model Training

1. Import required packages and data

In [1]:
import sys
print(sys.executable)

c:\Users\shiva\Desktop\Placement_Preparation\Git_repository\Churn_Modeling\Churn_Modeling\env\Scripts\python.exe


In [54]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
import scipy
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.metrics import classification_report,f1_score,recall_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier,AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from sklearn.svm import SVC
warnings.filterwarnings('ignore')

In [27]:
from src.components.data_ingestion import DataIngestion
data = DataIngestion()
csv_path= data.initiate_ingestion()

In [28]:
df = pd.read_csv(csv_path)

In [29]:
df.shape

(10000, 14)

In [30]:
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [31]:
df.drop(columns=['RowNumber','Surname','CustomerId'],inplace=True,axis=1)

In [32]:
print(f'Size of dataset after droping three columns{df.shape}')

Size of dataset after droping three columns(10000, 11)


In [33]:
X = df.drop(columns=['Exited'],axis = 1)
y = df['Exited']

In [34]:
numerical_features = [f for f in X.columns if X[f].dtype!='O']
categorical_features = [f for f in X.columns if f not in numerical_features]

print(f'{len(numerical_features)} numerical features are present in dataset and list is {numerical_features}')
print(f'{len(categorical_features)} categorical features are present in dataset and list is {categorical_features}')

8 numerical features are present in dataset and list is ['CreditScore', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'HasCrCard', 'IsActiveMember', 'EstimatedSalary']
2 categorical features are present in dataset and list is ['Geography', 'Gender']


In [35]:
scaler = StandardScaler()
encoder = OneHotEncoder()

preprocess = ColumnTransformer(
    [
        ("StandardScaler",scaler,numerical_features),
        ("OneHotEncoder",encoder,categorical_features)
    ]
)

In [36]:
X = preprocess.fit_transform(X)

In [38]:
X.shape

(10000, 13)

In [45]:
X_train, X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)

In [46]:
X_train.shape, X_test.shape

((8000, 13), (2000, 13))

In [55]:
models = {
    "AdaBoostClassifier":AdaBoostClassifier(),
    "CatBoostClassifier":CatBoostClassifier(verbose=False),
    "RandomForestClassifier":RandomForestClassifier(),
    "SVC": SVC(),
    "LogisticRegression":LogisticRegression(),
    "DecisionTreeClassifier": DecisionTreeClassifier(),
    "XGBClassifier":XGBClassifier()
}

In [57]:
model_list = {}


for i in range(len(list(models))):
    model = list(models.values())[i]
    model.fit(X_train,y_train)

   
    y_train_pred = model.predict(X_train)
    y_test_pred  = model.predict(X_test)
    
    #evaluate model on train and  test dataset
    train_f1_score = f1_score(y_train,y_train_pred)
    test_f1_score= f1_score(y_test,y_test_pred)
    train_recall_score = recall_score(y_train,y_train_pred)
    test_recall_score= recall_score(y_test,y_test_pred)

    model_list[list(models.keys())[i]] = [test_f1_score,test_recall_score]
    print(list(model_list.keys())[i])

    print("Model performance for training set")
    print(f'train recall score: {train_recall_score}')
    print(f'train f1 score: {train_f1_score}')
    
    print("__________________________________")

    print("Model performance on test dataset")
    print(f'test recall score: {test_recall_score}')
    print(f'test f1 score: {test_f1_score}')

    print('='*35)
    print('\n')


AdaBoostClassifier
Model performance for training set
train recall score: 0.4422141119221411
train f1 score: 0.5590157631680123
__________________________________
Model performance on test dataset
test recall score: 0.45038167938931295
test f1 score: 0.5557299843014128


CatBoostClassifier
Model performance for training set
train recall score: 0.6259124087591241
train f1 score: 0.7389587073608618
__________________________________
Model performance on test dataset
test recall score: 0.5038167938931297
test f1 score: 0.5972850678733032


RandomForestClassifier
Model performance for training set
train recall score: 0.9993917274939172
train f1 score: 0.9996957712199575
__________________________________
Model performance on test dataset
test recall score: 0.4707379134860051
test f1 score: 0.5808477237048666


SVC
Model performance for training set
train recall score: 0.40389294403892945
train f1 score: 0.5478547854785478
__________________________________
Model performance on test dataset

In [58]:
best_model = max(model_list, key=model_list.get)
best_score = model_list[best_model]

print("\nBest model:", best_model)
print("Best recall:", best_score)


Best model: CatBoostClassifier
Best recall: [0.5972850678733032, 0.5038167938931297]
